In [ ]:
!gdown --id 1SOb8wed7X8i0W_VaQsRqd1VZ11rnNU2_ # balanced
# !gdown --id 1TkHMa5rNULTDM_cKoE0oONiSxp1RpaDv # imbalanced

Downloading...
From: https://drive.google.com/uc?id=1SOb8wed7X8i0W_VaQsRqd1VZ11rnNU2_
To: /content/balanced_network-snapshot-025000.pkl
272MB [00:01, 145MB/s]


In [ ]:
!pip install ninja opensimplex

     |████████████████████████████████| 112kB 7.5MB/s 


# Closed Form Factorization method for StyleGAN2-ADA-Pytorch

In [ ]:
!git clone https://github.com/dvschultz/stylegan2-ada-pytorch

!python stylegan2-ada-pytorch/closed_form_factorization.py --out=/content/factor-test.pt --ckpt /content/balanced_network-snapshot-025000.pkl

# !python /content/stylegan2-pytorch/convert_weight.py --repo /content/stylegan2-ada-pytorch /content/balanced_network-snapshot-025000.pkl --gen --disc
!python stylegan2-ada-pytorch/apply_factor.py -i 0 -d 10 --seeds 5 --ckpt /content/balanced_network-snapshot-025000.pkl /content/factor-test.pt --output /content/test-vid/ --video

In [ ]:
import torch

factors = torch.load('/content/factor-test.pt')
eigenvecs = factors['eigvec'].numpy()

{'ckpt': '/content/balanced_network-snapshot-025000.pkl', 'eigvec': tensor([[-0.0218, -0.0716, -0.0196,  ...,  0.0363, -0.0055, -0.0016],
        [ 0.0239,  0.0685,  0.0584,  ..., -0.0013, -0.0339,  0.0226],
        [ 0.0221,  0.0047,  0.0290,  ...,  0.0233,  0.0459, -0.0008],
        ...,
        [ 0.0035, -0.0228, -0.0433,  ...,  0.0353,  0.0798,  0.0275],
        [ 0.0630, -0.0126,  0.0313,  ...,  0.0154, -0.0542,  0.1031],
        [ 0.0525,  0.0099, -0.0275,  ...,  0.0002,  0.0602,  0.0527]])}


# StyleGAN2-ADA-Pytorch -> StyleGAN2-Pytorch (*GANSpace input*)

In [44]:
# %cd /content
# !git clone https://github.com/alexferrante/stylegan2-ada-pytorch
# !git clone https://github.com/alexferrante/stylegan2-pytorch
%cd /content/stylegan2-ada-pytorch/

path_to_stylegan2_ada_pytorch_ckpt = '/content/balanced_network-snapshot-025000.pkl'
path_to_converted_pytorch_model = '/content/balanced_rosinality_model.pt'

# !python export_weights.py $path_to_stylegan2_ada_pytorch_ckpt $path_to_converted_pytorch_model

import torch
import pickle
import dnnlib 

device = torch.device('cuda')
with dnnlib.util.open_url(path_to_stylegan2_ada_pytorch_ckpt) as f:
  G = pickle.load(f)
  print(G)

converted_model = torch.load(path_to_converted_pytorch_model)
print(converted_model['g_ema'].keys())

/content/stylegan2-ada-pytorch
{'training_set_kwargs': {'class_name': 'training.dataset.ImageFolderDataset', 'path': './datasets/imbalanced_dataset_training', 'use_labels': False, 'max_size': 16214, 'xflip': True, 'resolution': 64}, 'G': Generator(
  (synthesis): SynthesisNetwork(
    (b4): SynthesisBlock(
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
    (b8): SynthesisBlock(
      (conv0): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
    (b16): SynthesisBlock(
      (conv0): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
 

In [ ]:
%cd /content/stylegan2-pytorch
!python generate.py --size 64 --channel_multiplier 1 --ckpt /content/balanced_rosinality_model.pt --truncation 0.5

/content
Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 378, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 378 (delta 2), reused 5 (delta 2), pack-reused 372
Receiving objects: 100% (378/378), 122.51 MiB | 21.90 MiB/s, done.
Resolving deltas: 100% (189/189), done.
/content/stylegan2-pytorch
  0% 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torchvision/utils.py:50: UserWarning: range will be deprecated, please use value_range instead.
  warnings.warn(warning)
100% 20/20 [00:00<00:00, 64.03it/s]


# Setup for GANSpace

In [ ]:
%cd /content
!git clone https://github.com/alexferrante/ganspace
%cd ganspace
!pip install fbpca pyopengltk glumpy
!git submodule update --init --recursive
!python -c "import nltk; nltk.download('wordnet')"
!wget https://files.pythonhosted.org/packages/5e/3f/5658c38579b41866ba21ee1b5020b8225cec86fe717e4b1c5c972de0a33c/pycuda-2019.1.2.tar.gz
!tar -xzf pycuda-2019.1.2.tar.gz
%cd /content/ganspace/pycuda-2019.1.2
!python configure.py --cuda-enable-gl --cuda-root=/usr/local/cuda
!make install
%cd "/content/ganspace"
%cd models/stylegan2/stylegan2-pytorch/op
!python -c "import torch; import upfirdn2d_op; import fused; print('OK')"
!python setup.py install

In [14]:
!pip install boto3 urllib3

In [65]:
# %cd /content
# !git clone https://github.com/alexferrante/ganspace
# %cd /content/ganspace/models/stylegan2
# !git clone https://github.com/alexferrante/stylegan2-pytorch
# %cd /content/ganspace

model = 'StyleGAN2'
model_class = 'dogs-bal'
num_components = 20

!python visualize.py --model $model --class $model_class --layer style --use_w
# !python interactive.py --model 'StyleGAN2' --class $model_class --layer=style -c $num_components --use_w
# !python interactive.py --model=StyleGAN2 --class=car --layer=style -n=1000000 -b=10000


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
[28.04 18:33] StyleGAN2, style, ipca
Feature shape: torch.Size([1, 512])
Not cached
[28.04 18:33] Computing stylegan2-dogs-bal_style_ipca_c80_n300000_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:289: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:263: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)
B=20, N=300000, dims=512, N/dims=585.9
Sampling latents: 100% 15100/15100 [00:18<00:00, 827.06it/s]
Fitting batches (NB=2000):

In [61]:
%cd /content/ganspace/models/stylegan2
!git clone https://github.com/harskish/stylegan2-pytorch

/content/ganspace/models/stylegan2
Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 316, done.
remote: Total 316 (delta 0), reused 0 (delta 0), pack-reused 316
Receiving objects: 100% (316/316), 114.15 MiB | 38.10 MiB/s, done.
Resolving deltas: 100% (158/158), done.


In [16]:
%cd /content/ganspace/models/stylegan2
!git clone https://github.com/alexferrante/stylegan2-pytorch

/content/ganspace/models/stylegan2
Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 378, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 378 (delta 2), reused 5 (delta 2), pack-reused 372
Receiving objects: 100% (378/378), 122.51 MiB | 33.08 MiB/s, done.
Resolving deltas: 100% (189/189), done.
